In [19]:
import evals
import torch
import logging
import transformers
import sys
import os
import nltk
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from trl import SFTConfig, SFTTrainer
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from peft import LoraConfig, PeftModel

In [4]:
"""
Initialize logger.
"""
transformers.utils.logging.set_verbosity_info()
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
    level=logging.INFO,
)

In [33]:
"""
Load model and tokenizer.
"""
# set seed before initializing model
set_seed(777)

# modified model & tokenizer code (due to transformers compatibility issue)
# model_id = "/home/javen/Projects/geb-1.3b"
# model_id = "GEB-AGI/geb-1.3b"
# model_id = "t5-small"
model_id = "sshleifer/distilbart-xsum-12-3"

# load model
# model = AutoModel.from_pretrained(model_id, trust_remote_code=True).bfloat16() #.cuda()
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, trust_remote_code=True).bfloat16()
# model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).bfloat16()

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# print(tokenizer.special_tokens)
tokenizer.add_special_tokens({'pad_token': '<pad>'})

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/javen/.cache/huggingface/hub/models--sshleifer--distilbart-xsum-12-3/snapshots/1d2bfbc16dcdd28720f9f1d37be764e5cc5c78c8/config.json
Model config BartConfig {
  "_name_or_path": "sshleifer/distilbart-xsum-12-3",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 3,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "eos_token_ids": [
    2
  ],
  "extra_pos_embeddings": 2,
  "force_bos_token_to_b

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /home/javen/.cache/huggingface/hub/models--sshleifer--distilbart-xsum-12-3/snapshots/1d2bfbc16dcdd28720f9f1d37be764e5cc5c78c8/pytorch_model.bin
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1
}

All model checkpoint weights were used when initializing BartForConditionalGeneration.

All the weights of BartForConditionalGeneration were initialized from the model checkpoint at sshleifer/distilbart-xsum-12-3.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.
Generation config file not found, using a generation config created from the model config.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

loading file vocab.json from cache at /home/javen/.cache/huggingface/hub/models--sshleifer--distilbart-xsum-12-3/snapshots/1d2bfbc16dcdd28720f9f1d37be764e5cc5c78c8/vocab.json
loading file merges.txt from cache at /home/javen/.cache/huggingface/hub/models--sshleifer--distilbart-xsum-12-3/snapshots/1d2bfbc16dcdd28720f9f1d37be764e5cc5c78c8/merges.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/javen/.cache/huggingface/hub/models--sshleifer--distilbart-xsum-12-3/snapshots/1d2bfbc16dcdd28720f9f1d37be764e5cc5c78c8/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/javen/.cache/huggingface/hub/models--sshleifer--distilbart-xsum-12-3/snapshots/1d2bfbc16dcdd28720f9f1d37be764e5cc5c78c8/tokenizer_config.json
loading configuration file config.json from cache at /home/javen/.cache/huggingface/hub/models--sshleifer--distilbart-xsum-12-3/snapshots/1d2bfbc16dcdd2872

1

In [23]:
"""
Load & clean WikiHow dataset.

https://huggingface.co/datasets/gursi26/wikihow-cleaned
https://github.com/mahnazkoupaee/WikiHow-Dataset
"""
def clean_dataset(dataset):
    df = pd.DataFrame(dataset)
    print(len(df))
    df = df.dropna()
    # df = df.iloc[:100]
    print(len(df))
    return Dataset.from_pandas(df)

def load_dataset():
    df = pd.read_csv("/home/javen/Projects/wikihow-cleaned/wikihow-cleaned.csv")
    # dataset = load_dataset("gursi26/wikihow-cleaned", split="train")
    dataset = Dataset.from_pandas(df)
    return dataset

# load dataset
dataset = load_dataset()
dataset = clean_dataset(dataset)

# split dataset
a = dataset.train_test_split(test_size=0.25)
b = a['test'].train_test_split(test_size=0.5)
dataset = DatasetDict({
    'train': a['train'],
    'test': b['test'],
    'valid': b['train']
})
print(dataset)

214293
213892
DatasetDict({
    train: Dataset({
        features: ['summary', 'title', 'text', '__index_level_0__'],
        num_rows: 192502
    })
    test: Dataset({
        features: ['summary', 'title', 'text', '__index_level_0__'],
        num_rows: 10695
    })
    valid: Dataset({
        features: ['summary', 'title', 'text', '__index_level_0__'],
        num_rows: 10695
    })
})


In [34]:
"""
Data tokenization & collation.
"""

# data collator
label_pad_token_id = tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8,
)

max_input_length = 256
max_target_length = 128

if model_id in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples['text']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # setup the tokenizer for targets
    labels = tokenizer(text_target=examples['summary'], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# apply tokenization
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/192502 [00:00<?, ? examples/s]

Map:   0%|          | 0/10695 [00:00<?, ? examples/s]

Map:   0%|          | 0/10695 [00:00<?, ? examples/s]

In [25]:
"""
Evaluation metrics.
"""
def compute_metrics(eval_preds):
    # prepare prediction data
    labels, preds = eval_preds.label_ids, eval_preds.predictions
    labels[labels == -100] = tokenizer.pad_token_id

    # decode
    preds_decoded = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels_decoded = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # calculate rouge scores
    rouge_scores = evals.calculate_rouge(labels_decoded, preds_decoded)

    # return metrics
    return rouge_scores

In [35]:
"""
Create Trainer.
"""
batch_size = 16
training_args = Seq2SeqTrainingArguments(
    learning_rate=5e-05,
    weight_decay=0.01,
    logging_dir='./logs',
    log_level='debug',
    output_dir='./output',
    save_steps=50,
    save_total_limit=3,
    use_cpu=True,
    eval_strategy='epoch',
    # eval_steps=10,
    num_train_epochs=1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    # greater_is_better=False,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['valid'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using auto half precision backend


In [ ]:
"""
Train model.
"""
training_results = trainer.train()

In [ ]:
"""
Generate summaries
"""
def summarize(model, text: str):
    # encode input text
    inputs = tokenizer(text, return_tensors="pt")
    inputs_length = len(inputs["input_ids"][0])
    print(inputs_length)

    # generate new text with model
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.1,
        )
        # print(outputs[0][inputs_length:])
        print(len(outputs[0]))

    # decode generated output text
    decoded = tokenizer.decode(
        # outputs[0][inputs_length:],
        outputs[0][inputs_length:],
        skip_special_tokens=True
    )
    return decoded

def generate_summaries(model, dataset, tokenizer, num_samples=5):
    summaries = []
    for i, example in enumerate(dataset):
        if i >= num_samples:
            break
        print(i)
        prompt = example['text']
        summary = summarize(model, prompt)
        summaries.append({'text': prompt, 'summary': summary})
    return summaries

In [ ]:
"""
Evaluate model.
"""
# summaries_hat = generate_summaries(model, dataset['test'], tokenizer)
# print(summaries_hat)